In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision

import time
import numpy as np

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import PIL

from easy_utils import ThreadDraw, LazyUI, make_torch_tensor_easy
import ipywidgets as w
from IPython.display import display, clear_output, Image

In [3]:
make_torch_tensor_easy()

done


In [4]:
W, H = 32, 32

bs = 12

seed = torch.rand(bs, 4, 4)
inp = torch.zeros(bs, 1, 16, 32)
inp[:, 0, 5:9, 11:15] = seed

ca_rule = nn.Sequential(
    nn.Conv2d(1, 3, (3, 3), stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(3, 10, (1, 1), stride=1, padding=0),
    nn.ReLU(),
    nn.Conv2d(10, 1, (1, 1), stride=1, padding=0),
    nn.Tanh(),
)

optim = torch.optim.Adam([
    *ca_rule.parameters(),
], lr=0.02)

In [5]:
history = []

In [6]:
DEVICE = 'cuda'
ca_rule = ca_rule.to(DEVICE)
inp = inp.to(DEVICE)
seed = seed.to(DEVICE)

In [7]:
# def run(return_dict):
#     while True:
#         A = np.random.rand(10, 10)
#         fig = plt.figure(dpi=100, figsize=(4, 4), tight_layout=True)

#         plt.imshow(np.random.rand(100, 100))

#         fig.canvas.draw()
#         plt.close()

#         X = np.array(fig.canvas.renderer.buffer_rgba())
#         return_dict['value'] = X

In [8]:
# import multiprocessing
# ma = multiprocessing.Manager()
# return_dict = ma.dict()
# p = multiprocessing.Process(target=run, args=(return_dict,))
# p.start()

In [9]:
# v = return_dict['value']
# PIL.Image.fromarray(v)

In [10]:
# p.terminate()

In [11]:
# ui_container = None
# def show_ui():
#     global ui_container
#     for i in range(1000):
#         ui_container = ui([
#             [tq.container],
#             [ui.plot(np.log(history[-200:]))],
#             [out.et.zoom(5)[:,0].cmap().grid(nrow=4, pad=5).pil[0]]
#         ]).display()

In [23]:
tq = tqdm(range(100), display=False)
for e in tq:
    out = inp
    for i in range(20):
        out = out + ca_rule(out)

    out_shape = out.shape
    read_area = out[:, :, :, out_shape[-1] // 2 :]
    read_seed = read_area[:, 0, 0:4, 11:15]

    loss = F.mse_loss(read_seed, seed)

    optim.zero_grad()
    loss.backward()
    optim.step()

    history.append(loss.item())
    tq.set_description(f"Loss: {loss.item():.5f}")
    
    LazyUI(lambda ui: ui([
        [tq.container],
        [ui.plot(np.log(history[-200:]))],
        [out.et.zoom(5)[:,0].cmap().grid(nrow=4, pad=5).pil[0]]
    ]).display())

In [20]:
td.t.is_alive()

True